In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
import PyPDF2
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

### 1. TẢI DỮ LIỆU

In [3]:
# Hàm trích xuất văn bản từ tệp PDF
def extract_text_from_resume(resume_path):
    """Trích xuất văn bản từ tệp PDF."""
    text = ""
    try:
        with open(resume_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Lỗi khi trích xuất văn bản từ {resume_path}: {e}")
    return text.strip()

In [4]:
# Hàm tải tất cả resume từ thư mục
def load_resumes(folder_path):
    """Tải resume từ thư mục và trích xuất văn bản."""
    resumes = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            resume_path = os.path.join(folder_path, filename)
            resume_text = extract_text_from_resume(resume_path)
            resumes.append(resume_text)
    return resumes

In [5]:
# Tải resume từ thư mục
resumes_folder = r'D:\BaiDoAnChuyenNganh3\accountant'
resumes = load_resumes(resumes_folder)
print(f"Đã tải {len(resumes)} resume.")

Đã tải 118 resume.


In [6]:
# Tải mô tả công việc từ tệp CSV
job_descriptions = pd.read_excel('D:\BaiDoAnChuyenNganh3\Automated-Resume-Ranking-System-main\csvfiles\crawlcv\DataFinal\GeneralCV_Example.xlsx')
print("Mẫu mô tả công việc:")
print(job_descriptions.head())

Mẫu mô tả công việc:
   ID                                         Resume_str  \
0   1           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1   2           HR SPECIALIST, US HR OPERATIONS      ...   
2   3           HR DIRECTOR       Summary      Over 2...   
3   4           HR SPECIALIST       Summary    Dedica...   
4   5           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


### 2. TIỀN XỬ LÝ DỮ LIỆU

In [7]:
# Hàm làm sạch văn bản
def clean_text(text):
    """Làm sạch văn bản: chuyển thành chữ thường, loại bỏ ký tự đặc biệt và khoảng trắng thừa."""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
# Kiểm tra giá trị thiếu trong cột Resume_str
print(f"Số giá trị thiếu trong Resume_str: {job_descriptions['Resume_str'].isna().sum()}")

# Xóa các dòng có giá trị thiếu trong Resume_str
job_descriptions = job_descriptions.dropna(subset=['Resume_str']).reset_index(drop=True)

# Đặt lại cột ID (tạo ID mới từ 1 đến số dòng)
job_descriptions['ID'] = range(1, len(job_descriptions) + 1)

# In số dòng sau khi xóa giá trị thiếu
print(f"Số dòng sau khi xóa giá trị thiếu: {len(job_descriptions)}")

Số giá trị thiếu trong Resume_str: 1
Số dòng sau khi xóa giá trị thiếu: 3973


In [9]:
# Làm sạch resume
cleaned_resumes = [clean_text(resume) for resume in resumes]
print("Mẫu resume đã làm sạch:", cleaned_resumes[0] if cleaned_resumes else "Không có resume.")

Mẫu resume đã làm sạch: accountant summary financial accountant specializing in financial planning reporting and analysis within the department of defense highlights account reconciliations resultsoriented financial reporting critical thinking accounting operations professional analysis of financial systems erp enterprise resource planning software excellent facilitator accomplishments served on a tiger team which identified and resolved general ledger postings in deams totaling b in accounting adjustments this allowed for the first successful fiscal yearend close for in collaboration with dfas europe developed an automated tool that identified duplicate obligations this tool allowed hq usafe to deobligate over m in duplicate obligations experience company name july to november accountant city state enterprise resource planning office ero in this position as an accountant assigned to the defense enterprise accounting and management system deams ero i was responsible for identifying and

In [10]:
# Làm sạch mô tả công việc
job_descriptions['Cleaned_Resume_str'] = job_descriptions['Resume_str'].apply(clean_text)
print("Mô tả công việc đã làm sạch:")
print(job_descriptions[['Resume_str', 'Cleaned_Resume_str']].head())

Mô tả công việc đã làm sạch:
                                          Resume_str  \
0           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1           HR SPECIALIST, US HR OPERATIONS      ...   
2           HR DIRECTOR       Summary      Over 2...   
3           HR SPECIALIST       Summary    Dedica...   
4           HR MANAGER         Skill Highlights  ...   

                                  Cleaned_Resume_str  
0  hr administratormarketing associate hr adminis...  
1  hr specialist us hr operations summary versati...  
2  hr director summary over years experience in r...  
3  hr specialist summary dedicated driven and dyn...  
4  hr manager skill highlights hr skills hr depar...  


In [11]:
# Hàm tiền xử lý với spaCy (lemmatization, loại bỏ stop words)
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    """Tiền xử lý văn bản: lemmatization, loại bỏ stop words và giữ lại chữ cái."""
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

In [12]:
# Tiền xử lý mô tả công việc
job_descriptions['Processed_Resume_str'] = job_descriptions['Resume_str'].apply(preprocess_text)
print("Mô tả công việc đã tiền xử lý:")
print(job_descriptions[['Resume_str', 'Processed_Resume_str']].head())

Mô tả công việc đã tiền xử lý:
                                          Resume_str  \
0           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1           HR SPECIALIST, US HR OPERATIONS      ...   
2           HR DIRECTOR       Summary      Over 2...   
3           HR SPECIALIST       Summary    Dedica...   
4           HR MANAGER         Skill Highlights  ...   

                                Processed_Resume_str  
0  hr administrator MARKETING associate hr admini...  
1  HR specialist hr operation Summary Versatile m...  
2  hr director Summary year experience recruiting...  
3  hr specialist Summary Dedicated driven dynamic...  
4  hr manager Skill Highlights hr skills hr Depar...  


### 3. VECTOR HÓA DỮ LIỆU

In [13]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(job_descriptions['Processed_Resume_str'])

In [14]:
# Chuyển ma trận TF-IDF thành DataFrame để dễ quan sát
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print("Mẫu ma trận TF-IDF:")
print(tfidf_df.head())

Mẫu ma trận TF-IDF:
    aa  aaa  aaae  aaahc  aaai  aaas  aac  aacc  aacn  aacsb  ...  µpg  \
0  0.0  0.0   0.0    0.0   0.0   0.0  0.0   0.0   0.0    0.0  ...  0.0   
1  0.0  0.0   0.0    0.0   0.0   0.0  0.0   0.0   0.0    0.0  ...  0.0   
2  0.0  0.0   0.0    0.0   0.0   0.0  0.0   0.0   0.0    0.0  ...  0.0   
3  0.0  0.0   0.0    0.0   0.0   0.0  0.0   0.0   0.0    0.0  ...  0.0   
4  0.0  0.0   0.0    0.0   0.0   0.0  0.0   0.0   0.0    0.0  ...  0.0   

   çatalhöyük  école  ética  étiquette  éxito  òwaterfalló  últimos  šarić  \
0         0.0    0.0    0.0        0.0    0.0          0.0      0.0    0.0   
1         0.0    0.0    0.0        0.0    0.0          0.0      0.0    0.0   
2         0.0    0.0    0.0        0.0    0.0          0.0      0.0    0.0   
3         0.0    0.0    0.0        0.0    0.0          0.0      0.0    0.0   
4         0.0    0.0    0.0        0.0    0.0          0.0      0.0    0.0   

   ﬁlm  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows x 3

### 4. HUẤN LUYỆN MÔ HÌNH

In [15]:
# Chuẩn bị dữ liệu huấn luyện
X = tfidf_matrix
y = job_descriptions['Category']

In [16]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Huấn luyện mô hình Logistic Regression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [19]:
# Dự đoán và đánh giá mô hình Logistic Regression
y_pred_lr = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Độ chính xác mô hình Logistic Regression: {lr_accuracy:.2f}")
print("\nBáo cáo phân loại Logistic Regression:")
print(classification_report(y_test, y_pred_lr))

Độ chính xác mô hình Logistic Regression: 0.58

Báo cáo phân loại Logistic Regression:
                               precision    recall  f1-score   support

                   ACCOUNTANT       0.54      0.89      0.68        28
         ACCOUNTANT & FINANCE       0.50      0.11      0.18         9
                     ADVOCATE       0.69      0.41      0.51        27
                  AGRICULTURE       0.33      0.08      0.13        12
                      APPAREL       0.56      0.56      0.56        16
                         ARTS       0.50      0.30      0.38        23
                   AUTOMOBILE       1.00      0.14      0.25         7
                     AVIATION       0.63      0.77      0.70        31
                      BANKING       0.48      0.72      0.58        18
        BILLING & COLLECTIONS       0.00      0.00      0.00         3
                          BPO       0.00      0.00      0.00         4
         BUSINESS DEVELOPMENT       0.36      0.59      0.44

d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
# Huấn luyện mô hình Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
# Dự đoán và đánh giá mô hình Random Forest
y_pred_rf = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Độ chính xác mô hình Random Forest: {rf_accuracy:.2f}")
print("\nBáo cáo phân loại Random Forest:")
print(classification_report(y_test, y_pred_rf))

Độ chính xác mô hình Random Forest: 0.62

Báo cáo phân loại Random Forest:
                               precision    recall  f1-score   support

                   ACCOUNTANT       0.42      0.93      0.58        28
         ACCOUNTANT & FINANCE       0.40      0.22      0.29         9
                     ADVOCATE       0.74      0.52      0.61        27
                  AGRICULTURE       0.80      0.33      0.47        12
                      APPAREL       0.67      0.62      0.65        16
                         ARTS       0.00      0.00      0.00        23
                   AUTOMOBILE       1.00      0.14      0.25         7
                     AVIATION       0.73      0.77      0.75        31
                      BANKING       0.45      0.50      0.47        18
        BILLING & COLLECTIONS       0.00      0.00      0.00         3
                          BPO       0.00      0.00      0.00         4
         BUSINESS DEVELOPMENT       0.35      0.53      0.42        17
 

d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\BaiDoAnChuyenNganh3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 5. LƯU MÔ HÌNH

In [22]:
# Lưu mô hình Logistic Regression
joblib.dump(lr_model, 'D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/model/lr_model.pkl')
print("Đã lưu mô hình Logistic Regression vào 'lr_model.pkl'.")

Đã lưu mô hình Logistic Regression vào 'lr_model.pkl'.


In [23]:
# Lưu mô hình Random Forest
joblib.dump(rf_model, 'D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/model/rf_model.pkl')
print("Đã lưu mô hình Logistic Regression vào 'rf_model.pkl'.")

Đã lưu mô hình Logistic Regression vào 'rf_model.pkl'.


In [24]:
# Lưu vectorizer 
joblib.dump(vectorizer, 'D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/Contacts/model/vectorizer.pkl')
print("Đã lưu vectorizer vào 'vectorizer.pkl'.")

Đã lưu vectorizer vào 'vectorizer.pkl'.
